Compute the total distance traveled by each participant (combining all the editions).

In [1]:
import os
import re
import glob
import logging
from haversine import haversine
from collections import Counter
import liegecolloquium

In [2]:
logger = logging.getLogger('distance-participants')
logger.setLevel(logging.DEBUG)
logging.debug("Start")

# Files and directories
We read directly the information from the files storing the coordinates.

In [3]:
datadir = "../data/"
participantfilelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-*.dat")))
outputdir = "../data/"
outputfile = os.path.join(outputdir, "ParticipantCountry.geojson")
logger.info("Working on {0} participant files".format(len(participantfilelist)))

INFO:distance-participants:Working on 39 participant files


# Main loop
For each edition, we loop over the participants and increment the list if they were not present before.     
The origin to compute the distance is Liège, no need to distinguish between Sart Tilman and city center.

In [12]:
participantlist = []
namelist = []
liegecoords = (50.5830803, 5.559063999999999)
maxdist = 0
maxparticipant = []
for participantfile in participantfilelist:
    logger.debug("Working on file {0}".format(os.path.basename(participantfile)))
    
    # Get year from file name using regex
    m = re.search(r"ParticipantList-(\d{4}).dat", participantfile)
    year = int(m.group(1))
    
    with open(participantfile, "r") as f:
        for line in f:
            l = line.rstrip().split('\t')
            participant = liegecolloquium.Participant(l[0], l[1], l[2], None, l[3])
            participant.lat = float(l[4])
            participant.lon = float(l[5])
            participant.replace_country()
            participant.abbrev_oneletter()
            
            # Compute distance to come to Liege
            participant.get_distance(origin=liegecoords)
            if participant.distance > maxdist:
                maxdist = participant.distance
                maxparticipant = [participant]
            elif participant.distance == maxdist:
                maxparticipant.append(participant)
            
            # Add to list if not present
            if participant.abbrev in namelist:
                participantindex = namelist.index(participant.abbrev)
                participantlist[participantindex].distance += participant.distance
                participantlist[participantindex].yearlist.append(year)
                # Increment distance and extend year list
            else:
                namelist.append(participant.abbrev)
                participant.yearlist.append(year)
                participantlist.append(participant)

DEBUG:distance-participants:Working on file ParticipantList-1970.dat
DEBUG:distance-participants:Working on file ParticipantList-1971.dat
DEBUG:distance-participants:Working on file ParticipantList-1972.dat
DEBUG:distance-participants:Working on file ParticipantList-1973.dat
DEBUG:distance-participants:Working on file ParticipantList-1974.dat
DEBUG:distance-participants:Working on file ParticipantList-1975.dat
DEBUG:distance-participants:Working on file ParticipantList-1976.dat
DEBUG:distance-participants:Working on file ParticipantList-1977.dat
DEBUG:distance-participants:Working on file ParticipantList-1978.dat
DEBUG:distance-participants:Working on file ParticipantList-1979.dat
DEBUG:distance-participants:Working on file ParticipantList-1980.dat
DEBUG:distance-participants:Working on file ParticipantList-1981.dat
DEBUG:distance-participants:Working on file ParticipantList-1982.dat
DEBUG:distance-participants:Working on file ParticipantList-1983.dat
DEBUG:distance-participants:Workin

In [13]:
logger.info(maxdist)
logger.info(maxparticipant)

INFO:distance-participants:17016.501088104655
INFO:distance-participants:[K Swalding (Australia): 17016.501088104655 km]


# Sort list according to distance

In [6]:
sortedParticipantList = sorted(participantlist, key=lambda participant: participant.distance, reverse=True)
sortedParticipantList[:10]

[P Chu (United States of America): 81394.61433764335 km,
 A Monreal (Mexico): 46523.22122995316 km,
 E Chassignet (United States of America): 45161.854422188815 km,
 D Salas de León (Mexico): 37177.771917873055 km,
 C Gibson (United States of America): 36704.12769935922 km,
 A Bah (Belgium): 35779.72414435937 km,
 L Goodman (United States of America): 35518.08303065271 km,
 Y Spitz (Belgium): 32484.165421106685 km,
 J Bye (Australia): 31790.124796643726 km,
 P Muller (United States of America): 29390.99579525198 km]

In [15]:
for p in sortedParticipantList[:10]:
    print(p.affiliation)

Naval Postgraduate School, Department of Oceanography - Naval Ocean Analysis and Prediction Laboratory
Centro de Investigación Científica y de Educación Superior
Rosenstiel School of Marine and Atmospheric Sciences
Centro de Investigación Científica y de Educación Superior
University of California San Diego
Université de Liège
Naval Underwater Systems Center
Université de Liège
School of Earth Sciences, The Flinders University of South Australia
Harvard University


## Longest distance travelled

In [11]:
logger.info("Longest distance travelled: {}".format(maxdist))
logger.info(maxname)
logger.info(maxname.affiliation)
logger.info(maxname.yearlist)

INFO:distance-participants:Longest distance travelled: 17016.501088104655
INFO:distance-participants:K Swalding (Australia): 17016.501088104655 km
INFO:distance-participants:University of Tasmania
INFO:distance-participants:[2006]


## Write into text file
Now we sort the participant by name.

In [21]:
sortedParticipantListName = sorted(participantlist, key=lambda participant: participant.name, reverse=False)

In [22]:
for p in sortedParticipantListName:
    p.write_name_country_years("../data/participant_years.dat")